In [29]:
import re, subprocess, pyodbc 
import numpy as np # linear algebra 
from sqlalchemy import func, create_engine
from config import mongoConn, connstr  
import pandas as pd  

In [30]:
engine = create_engine(connstr) 
def get_df(query):
    conn = engine.connect() 
    df = pd.read_sql(query,con=conn) 
    conn.close()
    return df

In [31]:
def cspan(IDTEXT): 
    span = 6
    if IDTEXT[0] == '0':
        return '6', '' 
    IDTEXT=IDTEXT.split('.')
    if len(IDTEXT) == 1:
        return '3','' 
    if len(IDTEXT) == 2:
        return '2',f'<td colspan="1">&nbsp</td>'   
    if len(IDTEXT) > 2:
        return '1',f'<td colspan="2">&nbsp</td>' 
    return str(span), ele
cspan('1.2.3')   

('1', '<td colspan="2">&nbsp</td>')

In [32]:
query = """ \
  	SELECT DISTINCT  
	Q_TypeCode AS FTPE,  IDTEXT,  SortPos, 
	PK_Question [{0}], 
	CASE WHEN ISNULL(IDTEXT, '') <> '' THEN
		REPLACE(REPLACE(IDTEXT, '.','_'), ' ','_') 
	ELSE
		'SDESC_' + CONVERT(NVARCHAR(9), SortPos)
	END AS [{1}], 
	CONVERT(NVARCHAR(5), PK_PickListType) + '' AS [{2}],
    '3' AS [{3}],
	CONVERT(NVARCHAR(9), PK_Question) + ' ' +  LEFT( REPLACE(REPLACE(QTEXT, ', Describe',''), 'If ','') , 15) AS [{4}],
	' r_idt_' + REPLACE(REPLACE(IDTEXT, '.','_'), ' ','_') +  ' ct_' +  Q_TypeCode  AS [{5}]
	FROM  vwQuestions WHERE PK_FORM LIKE '2022-A-HVA' AND SectionNum=1  ORDER BY SortPos ASC
"""  
df = get_df(query) 
df.fillna("",inplace=True)
df.replace(to_replace=[None], value=np.nan, inplace=True) 
df = df.astype(str)
df.to_csv(r'C:\Users\Tim\Documents\1802\HVA\form.csv', index=False)  
#df[0:2]

In [34]:
df = pd.read_csv(r'C:\Users\Tim\Documents\1802\HVA\form.csv', dtype=str)  
df.fillna("",inplace=True)
df = df.astype(str) 
#df[0:32]

In [37]:
st=''
for i,r in df.iterrows():
    tmp = f"C:\_som\_src\{r['FTPE']}.aspx"
    r0 = r['{0}']
    with open(tmp, 'r') as f:
        tmp = f.read()
    csp = cspan(r["IDTEXT"])
    tmp = re.sub('colspan="\d"','colspan="{3}"',tmp)
    tmp = tmp.replace("<!--{cols}-->", str(csp[1])) 
    tmp = tmp.replace("{3}", str(csp[0])) 
    tmp = tmp.replace("{0}", str(r["{0}"])) 
    tmp = tmp.replace("{1}", str(r["{1}"])) 
    tmp = tmp.replace("{2}", str(r["{2}"]))  
    tmp = tmp.replace("{4}", str(r["{0}"]) + ' ' + str(r["{4}"]))  
    tmp = tmp.replace("{5}", str(r["{5}"]))  
    st=st+tmp
with open(r'C:\Users\Tim\Documents\1802\HVA\page.aspx', 'w') as f:
    f.write(st) 

In [36]:
f1A=''
with open(fr'C:\_som\_src\1A.aspx', mode="r", encoding="utf-8") as f:
    f1A = f.read()  
f1A = f1A.replace("<!--BODY-->", st)
with open(r'D:\dev\CyberScope\CyberScopeBranch\CSwebdev\code\CyberScope\HVA\2022\2022_A_HVA_1B.aspx', 'w') as f:
    f.write(f1A) 
print( f1A )

<%@ Page Title="" Language="vb" AutoEventWireup="false" MasterPageFile="~/Form.Master" CodeBehind="2022_A_HVA_1B.aspx.vb" Inherits="CyberScope._2022_A_HVA_1B" %>

<%@ Register Src="../../CustomControls/CBlabel.ascx" TagName="CBlabel" TagPrefix="uc" %>
<%@ Register Src="../../CustomControls/CBoptionalLinks.ascx" TagName="CBoptionalLinks" TagPrefix="uc" %>
<%@ Register Src="../../CustomControls/CBButtPanel2.ascx" TagName="CBButtPanel" TagPrefix="uc" %>
<%@ Register Src="../../CustomControls/CBtext2.ascx" TagName="CBtext2" TagPrefix="uc" %>
<%@ Register Src="../../CustomControls/CBPickList2.ascx" TagName="CBPickList2" TagPrefix="uc" %>
<%@ Register Src="../../CustomControls/CbPickListMultiSelect.ascx" TagName="CbPickListMultiSelect" TagPrefix="uc" %>
<%@ Register Src="../../CustomControls/CBYesNo.ascx" TagName="CBYesNo" TagPrefix="uc" %>
<%@ Register Src="../../CustomControls/CBArtifact.ascx" TagPrefix="uc" TagName="CBArtifact" %>
<asp:Content ID="Content1" ContentPlaceHolderID="ContentPl